In [10]:
import tensorflow as tf
import tensorflow_datasets as tfds
import datetime

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [11]:
# get mnist from tensorflow_datasets
mnist = tfds.load("mnist", split =["train","test"], as_supervised=True)
train_ds = mnist[0]
val_ds = mnist[1]

In [12]:
# write function to create the dataset that we want
# "condition" expects of of two arguments: condition==0 being the comparison, and condition == 1 being the subtraction
def preprocess(data, batch_size, condition):
    
    
    # image should be float
    data = data.map(lambda x, t: (tf.cast(x, float), t))
    # image should be flattened
    data = data.map(lambda x, t: (tf.reshape(x, (-1,)), t))
    # image vector will here have values between 0 and 1
    data = data.map(lambda x,t: ((x/128.)-1, t))
    
    
    # we want to have two mnist images in each example
    # this leads to a single example being ((x1,y1),(x2,y2))
    zipped_ds = tf.data.Dataset.zip((data.shuffle(2000), 
                                     data.shuffle(2000)))
    
    if condition == 0:
        # use maping to change the target
        zipped_ds = zipped_ds.map(lambda x1, x2: (x1[0], x2[0], x1[1]+x2[1] >= 5))
        # transform boolean target to int
        zipped_ds = zipped_ds.map(lambda x1, x2, t: (x1,x2, tf.cast(t, tf.int32)))
    else:
        # use maping to change the target
        zipped_ds = zipped_ds.map(lambda x1, x2: (x1[0], x2[0], x1[1]-x2[1]))
        # transform target to float
        zipped_ds = zipped_ds.map(lambda x1, x2, t: (x1,x2, tf.cast(t, tf.float32)))
        
    
    # batch the dataset
    zipped_ds = zipped_ds.batch(batch_size)
    # prefetch
    zipped_ds = zipped_ds.prefetch(tf.data.AUTOTUNE)
    return zipped_ds

In [13]:
class TwinMNISTModel(tf.keras.Model):

    # 1. constructor
    # "condition" expects of of two arguments: condition==0 being the comparison, and condition == 1 being the subtraction
    # "opt" expects of of two arguments: opt==0 being the Adam optimizer, opt == 1 being the SGD optimizer, opt == 2 being RMSprop and opt == 3 being ADAGrad
    def __init__(self, condition, opt):
        # inherit functionality from parent class
        super().__init__()

        # metrics 
        if condition == 0:
            self.metrics_list = [tf.keras.metrics.Mean(name="loss"),
                                tf.keras.metrics.BinaryAccuracy()]
        else:
            self.metrics_list = [tf.keras.metrics.Mean(name="loss"),
                                 tf.keras.metrics.RootMeanSquaredError()]
        #optimizers
        if opt == 0:
            self.optimizer = tf.keras.optimizers.Adam()
        elif opt == 1: 
            self.optimizer = tf.keras.optimizers.SGD(0.01, 0.03)
        elif opt == 2: 
            self.optimizer = tf.keras.optimizers.RMSprop()
        elif opt == 3: 
            self.optimizer = tf.keras.optimizers.Adagrad()
        
        #loss functions 
        if condition == 0:
            self.loss_function = tf.keras.losses.BinaryCrossentropy()
        else:
            self.loss_function = tf.keras.losses.MeanSquaredError()
            
        # layers to encode the images (both layers used for both images)
        self.dense1 = tf.keras.layers.Dense(128, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(128, activation=tf.nn.relu)
        
        self.dense3 = tf.keras.layers.Dense(128, activation=tf.nn.relu)
        
        if condition == 0:
            self.out_layer = tf.keras.layers.Dense(1,activation=tf.nn.sigmoid)
        else:
            self.out_layer = tf.keras.layers.Dense(1,activation=None)
        
    # 2. call method (forward computation)
    def call(self, images, training=False):
        img1, img2 = images
        
        img1_x = self.dense1(img1)
        img1_x = self.dense2(img1_x)
        
        img2_x = self.dense1(img2)
        img2_x = self.dense2(img2_x)
        
        combined_x = tf.concat([img1_x, img2_x ], axis=1)
        combined_x = self.dense3(combined_x)
        return self.out_layer(combined_x)

    # 3. metrics property
    @property
    def metrics(self):
        return self.metrics_list
        # return a list with all metrics in the model

    # 4. reset all metrics objects
    def reset_metrics(self):
        for metric in self.metrics:
            metric.reset_states()

    # 5. train step method
    @tf.function
    def train_step(self, data):
        img1, img2, label = data
        
        with tf.GradientTape() as tape:
            output = self((img1, img2), training=True)
            loss = self.loss_function(label, output)
            
        gradients = tape.gradient(loss, self.trainable_variables)
        
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        
        # update the state of the metrics
        self.metrics[0].update_state(loss)
        for metric in self.metrics[1:]:
            metric.update_state(label, output)
            
        
        # return a dictionary with metric names as keys and metric results as values
        return {m.name : m.result() for m in self.metrics}

    # 6. test_step method
    @tf.function
    def test_step(self, data):
        img1, img2, label = data
        # same as train step (without parameter updates)
        output = self((img1, img2), training=False)
        loss = self.loss_function(label, output)
        self.metrics[0].update_state(loss)
        for metric in self.metrics[1:]:
            metric.update_state(label, output)
        
        return {m.name : m.result() for m in self.metrics}

In [14]:
def create_summary_writers(config_name):
    
    # Define where to save the logs
    # along with this, you may want to save a config file with the same name so you know what the hyperparameters were used
    # alternatively make a copy of the code that is used for later reference
    
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    train_log_path = f"logs/{config_name}/{current_time}/train"
    val_log_path = f"logs/{config_name}/{current_time}/val"

    # log writer for training metrics
    train_summary_writer = tf.summary.create_file_writer(train_log_path)

    # log writer for validation metrics
    val_summary_writer = tf.summary.create_file_writer(val_log_path)
    
    return train_summary_writer, val_summary_writer


In [15]:
import tqdm
def training_loop(model, train_ds, val_ds, start_epoch,
                  epochs, train_summary_writer, 
                  val_summary_writer, save_path):

    # 1. iterate over epochs
    for e in range(start_epoch, epochs):

        # 2. train steps on all batches in the training data
        for data in tqdm.tqdm(train_ds, position=0, leave=True):
            metrics = model.train_step(data)

        # 3. log and print training metrics

        with train_summary_writer.as_default():
            # for scalar metrics:
            for metric in model.metrics:
                    tf.summary.scalar(f"{metric.name}", metric.result(), step=e)
            # alternatively, log metrics individually (allows for non-scalar metrics such as tf.keras.metrics.MeanTensor)
            # e.g. tf.summary.image(name="mean_activation_layer3", data = metrics["mean_activation_layer3"],step=e)
        
        #print the metrics
        print([f"{key}: {value.numpy()}" for (key, value) in metrics.items()])
        
        # 4. reset metric objects
        model.reset_metrics()


        # 5. evaluate on validation data
        for data in val_ds:
            metrics = model.test_step(data)
        

        # 6. log validation metrics

        with val_summary_writer.as_default():
            # for scalar metrics:
            for metric in model.metrics:
                    tf.summary.scalar(f"{metric.name}", metric.result(), step=e)
            # alternatively, log metrics individually (allows for non-scalar metrics such as tf.keras.metrics.MeanTensor)
            # e.g. tf.summary.image(name="mean_activation_layer3", data = metrics["mean_activation_layer3"],step=e)
            
        print([f"val_{key}: {value.numpy()}" for (key, value) in metrics.items()])
        # 7. reset metric objects
        model.reset_metrics()
        
    # 8. save model weights if save_path is given
    if save_path:
        model.save_weights(save_path)

In [16]:
# open the tensorboard logs
%tensorboard --logdir logs/ --host localhost --port 8086

In [17]:
# 1. Apply preprocessing to datasets

train_ds = train_ds.take(1000)
val_ds = val_ds.take(100)

train_ds = preprocess(train_ds, batch_size=32, condition = 0) 
val_ds= preprocess(val_ds, batch_size=32, condition = 0) 


# 2. instantiate model
models = [TwinMNISTModel(condition = 0, opt = 0), TwinMNISTModel(condition = 0, opt = 1), TwinMNISTModel(condition = 0, opt = 2), TwinMNISTModel(condition = 0, opt = 3)]

# 3. choose a path to save the weights
save_path = "trained_model_RUN1"

# 4. pass arguments to training loop function
n = 1

for model in models: 
    train_summary_writer, val_summary_writer = create_summary_writers(config_name="RUN"+ str(n))
    n += 1
    
    training_loop(model=model,
        train_ds=train_ds,
        val_ds=val_ds,
        start_epoch=0,
        epochs=10,
        train_summary_writer=train_summary_writer,
        val_summary_writer=val_summary_writer,
        save_path=save_path)
    

100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:03<00:00,  8.79it/s]


['loss: 0.3523547053337097', 'binary_accuracy: 0.8669999837875366']
['val_loss: 0.37033623456954956', 'val_binary_accuracy: 0.8799999952316284']


100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 34.59it/s]


['loss: 0.2745150625705719', 'binary_accuracy: 0.8870000243186951']
['val_loss: 0.1771755814552307', 'val_binary_accuracy: 0.9200000166893005']


100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 50.74it/s]


...

['loss: 0.3430318832397461', 'binary_accuracy: 0.8650000095367432']
['val_loss: 0.3405369520187378', 'val_binary_accuracy: 0.8999999761581421']


100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 36.34it/s]


['loss: 0.3295154571533203', 'binary_accuracy: 0.8700000047683716']
['val_loss: 0.24748769402503967', 'val_binary_accuracy: 0.8899999856948853']


100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 52.65it/s]


['loss: 0.34645217657089233', 'binary_accuracy: 0.8529999852180481']
['val_loss: 0.24118660390377045', 'val_binary_accuracy: 0.8999999761581421']
